In [ ]:
!git clone https://github.com/johnjin0000/cs378_fp.git
%cd cs378_fp

Cloning into 'cs378_fp'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 67 (delta 14), reused 0 (delta 0), pack-reused 40
Unpacking objects: 100% (67/67), 38.61 KiB | 564.00 KiB/s, done.
/content/cs378_fp


In [ ]:
!ls
!pip install -r requirements.txt

adversarial_set.json  helpers.py  requirements.txt  test_contrast.json
contrast.csv	      README.md   run.py	    test_original.json
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.6/750.6 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 71.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 122.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.8 MB/s eta 0:00:00
   

In [ ]:
!pip3 install textattack[tensorflow]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.7/418.7 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 88.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 MB 18.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 72.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 kB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.1/373.

In [ ]:
# train model on snli dataset
!python3 run.py --do_train --task nli --dataset adversarial_set.json --model ./trained_model/ --output_dir ./trained_model/ --per_device_train_batch_size 8 --num_train_epochs 3

In [ ]:
# evaluate model on snli dataset, acc is 0.8917902708053589
!python3 run.py --do_eval --task nli --dataset snli --model ./trained_model/ --output_dir ./eval_output/

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from textattack.models.wrappers.huggingface_model_wrapper import HuggingFaceModelWrapper
from textattack.datasets import HuggingFaceDataset
from textattack.attack_recipes import TextFoolerJin2019
from textattack import Attacker, AttackArgs
from textattack.attack_results import SuccessfulAttackResult
import csv

dataset = HuggingFaceDataset("snli", None, "train", shuffle=True)
dataset.filter_by_labels_((0, 1, 2))
model = AutoModelForSequenceClassification.from_pretrained("./trained_model/", num_labels=3)
tokenizer = AutoTokenizer.from_pretrained("./trained_model", use_fast=True)
attack = TextFoolerJin2019.build(HuggingFaceModelWrapper(model, tokenizer))

attack_args = AttackArgs(
    num_examples = 150,
    disable_stdout=True,
)

attacker = Attacker(attack, dataset, attack_args)
results = attacker.attack_dataset()

# Open a CSV file in write mode
with open('attack_dataset.csv', 'w', newline='') as csvfile:
  # Create a CSV writer object
  writer = csv.writer(csvfile)
  write_lines = []
  for result in results:
    if isinstance(result, SuccessfulAttackResult):
      label = result.original_result.ground_truth_output
      premise, hypothesis = (result.original_result.attacked_text.text).split("\n")
      write_lines.append([premise, hypothesis, label])

  writer.writerow(["premise", "hypothesis", "label"])
  # Write data to CSV file line by line
  for line in write_lines:
    writer.writerow(line)


In [ ]:
# evaluate model on contrast set, 'eval_loss': 0.8511903285980225, 'eval_accuracy': 0.75
!python3 run.py --do_eval --task nli --dataset test_contrast.json --model ./trained_model/ --output_dir ./eval_output/

In [ ]:
# evaluate model on original set, 'eval_loss': 0.40389466285705566, 'eval_accuracy': 0.8700000047683716,
!python3 run.py --do_eval --task nli --dataset test_original.json --model ./trained_model/after_training/ --output_dir ./eval_output/

In [ ]:
def prepare_dataset_nli(examples, tokenizer, max_seq_length=None):
    max_seq_length = tokenizer.model_max_length if max_seq_length is None else max_seq_length

    tokenized_examples = tokenizer(
        examples['premise'],
        examples['hypothesis'],
        truncation=True,
        max_length=max_seq_length,
        padding='max_length'
    )

    tokenized_examples['label'] = examples['label']
    return tokenized_examples

from transformers import Trainer, AutoModelForSequenceClassification, AutoTokenizer
import datasets

# Load model from checkpoint
model = AutoModelForSequenceClassification.from_pretrained("./trained_model/after_training/", num_labels=3)
tokenizer = AutoTokenizer.from_pretrained("./trained_model/after_training/", use_fast=True)
trainer = Trainer(model=model, tokenizer=tokenizer)

# Test original dataset
orig_dataset = datasets.load_dataset('json', data_files="test_original.json", field="data")
orig_eval_dataset = orig_dataset["train"]
orig_eval_dataset_featurized = orig_eval_dataset.map(
  lambda exs: prepare_dataset_nli(exs, tokenizer, 128),
  batched=True,
  num_proc=2,
  remove_columns=orig_eval_dataset.column_names
)
orig_pred, orig_labels, metrics = trainer.predict(orig_eval_dataset_featurized)

# Test contrast dataset
cont_dataset = datasets.load_dataset('json', data_files="test_contrast.json", field="data")
cont_eval_dataset = cont_dataset["train"]
cont_eval_dataset_featurized = cont_eval_dataset.map(
  lambda exs: prepare_dataset_nli(exs, tokenizer, 128),
  batched=True,
  num_proc=2,
  remove_columns=cont_eval_dataset.column_names
)
cont_pred, cont_labels, metrics = trainer.predict(cont_eval_dataset_featurized)

# Test Old Model before adversarial training
model = AutoModelForSequenceClassification.from_pretrained("./trained_model/", num_labels=3)
tokenizer = AutoTokenizer.from_pretrained("./trained_model/", use_fast=True)
trainer = Trainer(model=model, tokenizer=tokenizer)

old_cont_pred, old_cont_labels, old_metrics = trainer.predict(cont_eval_dataset_featurized)

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix
cont_pred_argmax = np.argmax(cont_pred, axis=1)
print(cont_pred_argmax)
print(cont_labels)
cm = confusion_matrix(cont_pred_argmax, cont_labels)
print(cm)

[2 2 0 2 2 2 2 0 2 2 2 1 2 1 2 0 2 1 2 2 2 1 2 2 2 2 1 2 1 2 1 1 0 2 2 2 2
 1 1 2 2 0 2 0 2 2 2 2 2 1 1 2 1 1 2 2 1 2 2 2 2 1 2 1 0 0 0 2 2 2 2 2 2 0
 2 2 1 2 2 0 2 1 1 1 2 1 0 0 0 0 0 1 0 0 2 2 1 0 1 1]
[2 2 1 2 2 2 2 1 2 1 2 1 2 2 2 2 2 1 2 2 2 1 2 2 2 2 1 2 1 2 1 1 2 2 2 2 2
 1 1 2 2 0 2 0 2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 0 0 0 0 2 2 2 2 2 2 0
 0 0 1 1 0 0 1 1 1 1 1 1 0 1 1 0 0 1 1 0 1 1 1 1 1 1]
[[11  6  2]
 [ 2 18  6]
 [ 3  6 46]]


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix
old_cont_pred_argmax = np.argmax(old_cont_pred, axis=1)
print(old_cont_pred_argmax)
print(old_cont_labels)
cm2 = confusion_matrix(old_cont_pred_argmax, old_cont_labels)
print(cm)

[2 2 0 2 2 2 0 0 2 2 2 1 2 1 2 0 2 1 2 2 2 1 2 2 2 2 1 2 1 2 1 0 0 0 2 0 2
 1 1 2 2 0 2 0 2 1 2 2 2 1 1 2 1 1 2 2 1 2 1 1 2 1 2 1 0 0 0 1 2 2 1 2 2 0
 2 2 1 2 1 0 2 1 1 1 2 1 0 0 0 0 0 1 0 0 2 2 1 0 1 1]
[2 2 1 2 2 2 2 1 2 1 2 1 2 2 2 2 2 1 2 2 2 1 2 2 2 2 1 2 1 2 1 1 2 2 2 2 2
 1 1 2 2 0 2 0 2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 0 0 0 0 2 2 2 2 2 2 0
 0 0 1 1 0 0 1 1 1 1 1 1 0 1 1 0 0 1 1 0 1 1 1 1 1 1]
[[11  7  5]
 [ 3 17 11]
 [ 2  6 38]]


In [ ]:
both_correct = 0
old_correct = 0
new_correct = 0
both_wrong = 0

for i in range(len(cont_labels)):
  if old_cont_pred_argmax[i] == cont_labels[i] and cont_pred_argmax[i] == cont_labels[i]:
    both_correct += 1
  elif old_cont_pred_argmax[i] == cont_labels[i]:
    old_correct += 1
  elif cont_pred_argmax[i] == cont_labels[i]:
    new_correct += 1
  else:
    both_wrong += 1

print(both_correct, old_correct, new_correct, both_wrong)


66 0 9 25


In [ ]:
selected_examples = []
for i in range(len(cont_labels)):
  if old_cont_pred_argmax[i] != cont_labels[i] and cont_pred_argmax[i] == cont_labels[i]:
    selected_examples.append((i, old_cont_pred_argmax[i]))

import json

with open('test_contrast.json', 'r') as jsonfile:
    # Write data as JSON
    contrast_exs = json.load(jsonfile)

for idx in selected_examples:
  print(contrast_exs["data"][idx[0]])
  print("old predicted", idx[1])


{'premise': 'two boys reading superhero books', 'hypothesis': 'three boys reading a piece of literature.', 'label': 2}
old predicted 0
{'premise': 'Two women walk down a sidewalk along a busy street in a downtown area.', 'hypothesis': 'The beutiful women were walking downtown', 'label': 1}
old predicted 0
{'premise': 'The girls walk down the street.', 'hypothesis': 'Girls walk down the alley.', 'label': 2}
old predicted 0
{'premise': "A man of the cloth puts a black substance on a man's forehead.", 'hypothesis': 'The man puts something on the other mans nose.', 'label': 2}
old predicted 0
{'premise': 'The tattooed basketball player is about to dunk the ball.', 'hypothesis': 'The player has a book that says mother.', 'label': 2}
old predicted 1
{'premise': 'A basketball player with green shoes is dunking the ball in the net while the arena crowd looks on.', 'hypothesis': 'A famous NBA player is playing street ball in front of no one.', 'label': 2}
old predicted 1
{'premise': 'A frowning

In [ ]:
selected_examples = []
for i in range(len(cont_labels)):
  if old_cont_pred_argmax[i] != cont_labels[i] and cont_pred_argmax[i] != cont_labels[i]:
    selected_examples.append((i, old_cont_pred_argmax[i], cont_pred_argmax[i]))

import json

with open('test_contrast.json', 'r') as jsonfile:
    # Write data as JSON
    contrast_exs = json.load(jsonfile)

for idx in selected_examples:
  print(contrast_exs["data"][idx[0]])
  print("old predicted", idx[1])
  print("new predicted", idx[2])

{'premise': 'A woman with a green headscarf, blue shirt and a very big grin.', 'hypothesis': 'The woman is very emotional.', 'label': 1}
old predicted 0
new predicted 0
{'premise': 'A statue at a museum that no seems to be looking at.', 'hypothesis': 'There is a statue that not many people seem to be know about.', 'label': 1}
old predicted 0
new predicted 0
{'premise': 'A blond-haired doctor and her African american assistant looking threw new medical manuals.', 'hypothesis': 'A african american doctor is looking at a book', 'label': 1}
old predicted 2
new predicted 2
{'premise': 'Male in a blue jacket decides to lay in the grass.', 'hypothesis': 'The guy wearing a blue jacket is laying on the blue grass', 'label': 2}
old predicted 1
new predicted 1
{'premise': 'A woman wearing a ball cap squats down to touch the cracked earth.', 'hypothesis': 'A squatting woman wearing a hat hitting the ground.', 'label': 2}
old predicted 0
new predicted 0
{'premise': 'Many people standing outside of 

In [ ]:
orig_pred_argmax = np.argmax(orig_pred, axis=1)
print(orig_pred_argmax)
print(orig_labels)
cm = confusion_matrix(orig_pred_argmax, orig_labels)
print(cm)

[0 0 0 0 0 0 0 0 0 2 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 2 0 0
 0 0 0 1 1 1 1 0 1 0 1 2 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 2 1 1 1 2 0 1 1 1 2
 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
[[36  3  0]
 [ 1 26  2]
 [ 3  4 25]]
